# 🚀 Quick Start - Panel Dashboard

**最新版本**: Phase 3.0.1 - Physical Isolation + Self-Documenting  
**特性**: 物理隔离 API (`data_values`) + AI 业务备注 + 自动 HTML/PDF 导出

---

## 📋 本 Notebook 的步骤

1. **初始化环境** - 加载框架 + CSS 宽度修复
2. **加载数据** - 从 Parquet 文件加载
3. **生成 AI Prompt** - 包含 v3.0 物理隔离规范（复制给 AI）
4. **选择维度** - 选择分析维度
5. **创建仪表盘** - 自动生成布局
6. **生成分析代码** - 粘贴 AI 生成的逻辑
7. **导出 PDF 报告** - 自动提取 AI 备注生成专业文档


## 📦 Step 1: 初始化环境

加载框架 + CSS 宽度修复（让图表占满屏幕）

In [1]:
from IPython.display import HTML, display
import polars as pl
import plotly.express as px
import panel as pn
from src.session import DataSession
from src.dashboard import PanelDashboardBuilder
from src.utils import print_markdown_table

# 1. 移除 JupyterLab 宽度限制（关键！）
display(HTML("<style>.jp-Notebook { --jp-notebook-max-width: 100% !important; }</style>"))

# 2. 初始化 Panel（全局设置 stretch_width）
pn.extension('plotly', sizing_mode='stretch_width')

print("✅ 环境初始化完成")
print("📚 框架版本: Phase 3.0.1 - Physical Isolation (v3.0.1)")


✅ 环境初始化完成
📚 框架版本: Phase 3.0.1 - Physical Isolation (v3.0.1)


## 📂 Step 2: 加载数据

从 Parquet 文件加载数据

In [2]:
# 创建数据会话
session = DataSession()

# 加载数据（替换为你的数据文件）
session.load("insurance_data_cleaned", alias="df")  # 或 session.load("your_data.parquet", alias="df")

# 验证数据
print(f"✅ 数据已加载: {df_df.height:,} 行")
print_markdown_table(df_df.head(5))


✅ 已加载: df_df (insurance_data_cleaned)
✅ 数据已加载: 776,739 行


|   业务年度 |   机构代码 | 机构名称                         |   业务险种 |                 保单号 |   危险单位号 | 支付币别   | 业务来源   | 是否联共保   |   联共保我司占比 |         地址代码 | 标的地址或船名                                                   | 被保险人名称               | 保险起期   | 保险止期   |   风险代码 | 风险代码描述                                         |   风险等级 |      总保额 |   总保费 |   总保费_原币 |   手续费 |   总已决赔款 |   总未决赔款 |    自留保额 |   自留保费 |   自留已决 |   自留未决 |   附加自留保额 |   附加自留保费 |   附加自留已决 |   附加自留未决 |   协议分保额 |   协议分保费 |   协议摊回费用 |   协议已决 |   协议未决 |   成数分保额 |   成数分保费 |   成数摊回费用 |   成数已决 |   成数未决 |   溢额分保额 |   溢额分保费 |   溢额摊回费用 |   溢额已决 |   溢额未决 |   OPENCOVER分保额 |   OPENCOVER分保费 |   OPENCOVER摊回费用 |   OPENCOVER已决 |   OPENCOVER未决 |   临分分保额 |   临分分保费 |   临分摊回费用 |   临分已决 |   临分未决 | 占用性质名称   | 行业类别名称   | 境内境外   |   分入手续费率 |   分入手续费 |   分入出单费比例 |   分入出单费 |   分入经纪费率 |   分入经纪费 | 分入经纪人名称   | 直保代理人名称                           | 归属机构   |
|-----------:|-----------:|:---------------------------------|-----------:|-----------------------:|-------------:|:-----------|:-----------|:-------------|-----------------:|-----------------:|:-----------------------------------------------------------------|:---------------------------|:-----------|:-----------|-----------:|:-----------------------------------------------------|-----------:|------------:|---------:|--------------:|---------:|-------------:|-------------:|------------:|-----------:|-----------:|-----------:|---------------:|---------------:|---------------:|---------------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|------------------:|------------------:|--------------------:|----------------:|----------------:|-------------:|-------------:|---------------:|-----------:|-----------:|:---------------|:---------------|:-----------|---------------:|-------------:|-----------------:|-------------:|---------------:|-------------:|:-----------------|:-----------------------------------------|:-----------|
|       2024 |   44030033 | 深圳分公司工商保险业务部         |       0102 | 3010220244403000000577 |            1 | CNY        | 直接业务   | 非联共保     |            100   | 4403003300008323 | 广东省-深圳市-福田区滨河大道与益田路交界东南皇庭世纪花园1号楼14A | 崔鹏                       | 2024-04-23 | 2027-07-22 |      24107 | 中行渠道财产险类一般行业                             |          1 | 7.8329e+06  |  1921.27 |       1921.27 |     0    |            0 |            0 | 7.8329e+06  |    1921.27 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 综合商业类     |                | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  |                                          | 深圳       |
|       2016 |   37070004 | 山东分公司潍坊中心支公司银保团队 |       0101 | 3010120163707000000006 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3707000300000714 | 山东省潍坊市滨海经济开发区新海大街99号                           | 潍坊友容实业有限公司       | 2016-01-06 | 2017-01-05 |      11160 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 3.97982e+07 | 27858.8  |      27858.8  |  4178.81 |            0 |            0 | 3.97982e+07 |   27858.8  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 建筑安装业     | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司潍坊分行             | 山东       |
|       2023 |   31030001 | 上海自贸试验区分公司经理室       |       0104 | 3010420233103000001350 |            1 | CNY        | 经纪业务   | 从共         |             10.5 | 3100001700042357 | 新疆维吾尔自治区哈密地区哈密市伊州区                             | 中国广核集团               | 2023-02-01 | 2023-09-30 |      23056 | 市场渠道共保财产险类电厂及水电站（不含联合循环电厂） |          2 | 1.33116e+08 | 21421.9  |      21421.9  |  3406.08 |            0 |            0 | 1.2e+08     |   19311.2  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  1.31159e+07 |      2110.69 |         675.42 |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 二级工业       | 其他电力生产   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中广核保险经纪有限责任公司               | 上海       |
|       2020 |   33070007 | 浙江分公司金华中心支公司银保团队 |       0101 | 3010120203307000000114 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3307000800003824 | 浙江省金华市兰溪市兰江街道毕家村                                 | 兰溪市三江仓储发展有限公司 | 2020-05-23 | 2021-05-22 |      20107 | 中行渠道财产险类一般行业                             |          3 | 1.23e+06    |  1160.38 |       1160.38 |     0    |            0 |            0 | 1.23e+06    |    1160.38 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 工业           | 内河货物运输   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司兰溪支行             | 浙江       |
|       2011 |   32010004 | 江苏分公司南京营业部银保团队     |       0102 |  301022010320100000012 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3200001000000077 | 江苏省南京市太平北路106号第四至七层                              | 宏图三胞高科技术有限公司   | 2011-01-01 | 2011-12-31 |      10383 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 2.95e+07    | 14750    |      14750    |  2360    |            0 |            0 | 2.832e+07   |   14160    |          0 |          0 |              0 |              0 |              0 |              0 |     1.18e+06 |          590 |          241.9 |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 其他科技服务   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行南京地区鼓楼支行公司板块公司业务 | 江苏       |

## 📋 Step 3: 生成完整 AI Prompt

**运行这个 cell，然后复制全部输出给 AI**

包含数据结构信息、v3.0 物理隔离规范以及分析自我叙述要求。

In [3]:
import os

# 自动找到项目根目录
def find_project_root():
    current = os.path.abspath('.')
    while current != '/':
        if os.path.exists(os.path.join(current, 'src')): return current
        current = os.path.dirname(current)
    return os.path.abspath('.')

step4_path = os.path.join(find_project_root(), 'notebooks/templates/step4_standalone.py')

if os.path.exists(step4_path):
    exec(open(step4_path).read())
else:
    print("📋 请根据 main.md 指导 AI 进行分析。")
    print("物理隔离规范 (v3.0.1):")
    print("- 必须使用: filters = dashboard.data_values")
    print("- 必须使用: agg_dim = dashboard.widgets['_aggregation_dimension'].value")
    print("- 必须包含: [REPORT_METADATA] 文档块")


📋 **复制以下所有内容给 AI**

## 📊 数据结构

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float6

## 🎯 Step 4: 选择分析维度

选择你想要分析的维度字段

In [4]:
selected_dimensions = [
    '业务年度',
    '业务险种',
    '归属机构' 
]
print(f"✅ 已选择维度: {selected_dimensions}")


✅ 已选择维度: ['业务年度', '业务险种', '归属机构']


## 📊 Step 5: 创建 Panel 仪表盘

自动创建仪表盘控件，并打印引导 AI 的同步指令。

In [5]:
dashboard = PanelDashboardBuilder.from_data(
    df_df,
    dimensions=selected_dimensions,
    title="📊 数据分析仪表盘 (v3.0.1)"
)

print("\n📝 给 AI 的提示:")
print("=" * 80)
print("请严格遵守 main.md 中的 v3.0.1 规范。")
print("1. 物理隔离：严禁遍历 .widgets，必须使用 .data_values 获取过滤值。")
print("2. 自我叙述：必须在 update_dashboard 函数的 Docstring 中包含 [REPORT_METADATA] 块，总结分析背景与逻辑。")
print(f"已选择维度: {selected_dimensions}")
print("需求: [在这里描述你的分析需求]")
print("=" * 80)


🎨 从数据创建仪表盘: 📊 数据分析仪表盘 (v3.0.1)
📊 数据维度: 776,739 行 × 69 列
🔧 配置维度字段: 业务年度, 业务险种, 归属机构



  ✅ 业务年度: MultiChoice (25 个选项 + 全选, 默认选 1 个)
  ✅ 业务险种: MultiChoice (26 个选项 + 全选, 默认选 1 个)
  ✅ 归属机构: MultiChoice (30 个选项 + 全选, 默认选 1 个)

  ✅ 聚合维度选择器: Select (3 个维度可选, 默认: 业务年度)

✅ 仪表盘控件创建完成 (4 个控件)
💡 下一步: 使用 dashboard.set_update_function(your_function)

📄 等效代码（可复制用于自定义）:

```python
import panel as pn
import polars as pl

dashboard = PanelDashboardBuilder(title="📊 数据分析仪表盘 (v3.0.1)")

# 示例: 业务年度
widget = pn.widgets.Select(name='业务年度', options=[...])
dashboard.widgets['业务年度'] = widget

# 示例: 业务险种
widget = pn.widgets.Select(name='业务险种', options=[...])
dashboard.widgets['业务险种'] = widget

# 定义更新函数
@pn.depends(*dashboard.widgets.values())
def update(*args):
    # 你的分析逻辑
    return fig

dashboard.set_update_function(update)
dashboard.show()  # Jupyter 显示
dashboard.save('output.html')  # 导出 HTML
```



📝 给 AI 的提示:
请严格遵守 main.md 中的 v3.0.1 规范。
1. 物理隔离：严禁遍历 .widgets，必须使用 .data_values 获取过滤值。
2. 自我叙述：必须在 update_dashboard 函数的 Docstring 中包含 [REPORT_METADATA] 块，总结分析背景与逻辑。
已选择维度: ['业务年度', '业务险种', '归属机构']

## 🤖 Step 6: AI 生成分析逻辑

**在这里粘贴 AI 生成的代码**

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import polars as pl
import panel as pn
from src.utils import print_markdown_table

# ⚠️ 确保 dashboard 对象已存在
# dashboard = PanelDashboardBuilder.from_data(...)

@pn.depends(*dashboard.widgets.values())
def update_dashboard(*args):
    """
    [REPORT_METADATA]
    ### 1. 业务背景
    对溢额合约进行“投入(保费)-产出(损益)”的全景分析。区分“超法定自留”大额业务与“风险分散”常规业务，评估分保结构的合理性。

    ### 2. 分析逻辑
    - **分类标准**: (自留+附加+溢额)保额 ≥ 4.5亿 标记为 '超法定自留'，否则为 '风险分散'。
    - **图表结构**: 
        - 上图 (保费): 溢额分保费 (堆叠柱状图)，展示分出规模构成。
        - 下图 (损益): 再保承保利润 (分组柱状图)，展示合约盈亏结果。
    - **配色风格**: 冷色系 (靛青/天蓝)。
    [/REPORT_METADATA]
    """
    # 1. 🛡️ 物理隔离
    filters = dashboard.data_values
    agg_dim = dashboard.widgets['_aggregation_dimension'].value
    
    # --- A. 动态过滤 ---
    filtered = df_df
    for col, val in filters.items():
        if isinstance(val, list):
            if '全选' not in val:
                filtered = filtered.filter(pl.col(col).is_in(val))
        elif val != '全选':
            filtered = filtered.filter(pl.col(col) == val)
            
    # --- B. 核心逻辑：分类与计算 ---
    THRESHOLD = 450_000_000
    
    # 预计算
    calculated = filtered.select([
        pl.col(agg_dim),
        # 1. 分类
        pl.when(
            (pl.col('自留保额').fill_null(0) + 
             pl.col('附加自留保额').fill_null(0) + 
             pl.col('溢额分保额').fill_null(0)) >= THRESHOLD
        ).then(pl.lit('超法定自留'))
         .otherwise(pl.lit('风险分散'))
         .alias('业务分类'),
         
        # 2. 保费 (分出规模)
        pl.col('溢额分保费').fill_null(0).alias('保费_raw'),
        
        # 3. 损益 (再保利润)
        (pl.col('溢额分保费').fill_null(0) - 
         (pl.col('溢额已决').fill_null(0) + pl.col('溢额未决').fill_null(0)) - 
         pl.col('溢额摊回费用').fill_null(0)
        ).alias('损益_raw')
    ])
    
    # --- C. 聚合分析 ---
    analysis = calculated.group_by([agg_dim, '业务分类']).agg([
        pl.col('保费_raw').sum().alias('保费额'),
        pl.col('损益_raw').sum().alias('损益额'),
        pl.len().alias('保单数')
    ]).sort(agg_dim)
    
    # --- D. 单位转换 (万元) ---
    unit = "万元"
    div = 10_000
    
    analysis = analysis.with_columns([
        (pl.col('保费额') / div).alias('保费'),
        (pl.col('损益额') / div).alias('损益')
    ])
    
    # 转 Pandas 用于绘图
    df_pd = analysis.to_pandas()
    df_super = df_pd[df_pd['业务分类'] == '超法定自留']
    df_normal = df_pd[df_pd['业务分类'] == '风险分散']
    
    # --- E. 可视化 (双子图 - 冷色系) ---
    fig = make_subplots(
        rows=2, cols=1, 
        shared_xaxes=True,
        vertical_spacing=0.12,
        subplot_titles=(f"📈 分出保费规模 ({unit})", f"📉 再保承保利润 ({unit})")
    )

    # 🎨 冷色系配色
    COLOR_SUPER = '#3F51B5'  # 靛青色
    COLOR_NORMAL = '#03A9F4' # 天蓝色

    # === Row 1: 保费 (堆叠) ===
    fig.add_trace(
        go.Bar(x=df_super[agg_dim], y=df_super['保费'], name='超法定-保费', 
               marker_color=COLOR_SUPER, opacity=0.8, legendgroup='super'),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(x=df_normal[agg_dim], y=df_normal['保费'], name='风险分散-保费', 
               marker_color=COLOR_NORMAL, opacity=0.8, legendgroup='normal'),
        row=1, col=1
    )

    # === Row 2: 损益 (分组) ===
    fig.add_trace(
        go.Bar(x=df_super[agg_dim], y=df_super['损益'], name='超法定-损益', 
               marker_color=COLOR_SUPER, legendgroup='super', showlegend=False),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=df_normal[agg_dim], y=df_normal['损益'], name='风险分散-损益', 
               marker_color=COLOR_NORMAL, legendgroup='normal', showlegend=False),
        row=2, col=1
    )
    
    # 0 线参考
    fig.add_hline(y=0, line_dash="solid", line_color="#90A4AE", row=2, col=1)

    # 布局设置
    fig.update_layout(
        height=700,
        autosize=True, #
        barmode='group',
        hovermode="x unified",
        margin=dict(l=40, r=40, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    fig.update_yaxes(showgrid=True, gridcolor='#ECEFF1')

    # --- F. Markdown 报告 (含合计行) ---
    print(f"## ❄️ 溢额合约全景报告 (冷色系版)\n")
    
    # 构造透视表
    display_df = analysis.pivot(
        index=agg_dim, 
        on='业务分类', 
        values=['保费', '损益'],
        aggregate_function='sum'
    ).sort(agg_dim)
    
    # 整理列顺序
    try:
        cols = sorted(display_df.columns)
        display_df = display_df.select([agg_dim] + [c for c in cols if c != agg_dim])
    except:
        pass

    # 🔥 核心更新：计算并添加合计行
    # 1. 识别数值列（排除分组列）
    numeric_cols = [c for c in display_df.columns if c != agg_dim]
    
    # 2. 构建合计行 DataFrame
    total_row = display_df.select([
        pl.lit('**合计**').alias(agg_dim), # 第一列显示"合计"
        *[pl.col(c).sum().alias(c) for c in numeric_cols] # 其他列求和
    ])
    
    # 3. 拼接到原表底部
    final_table = pl.concat([display_df, total_row])
    
    # 打印全量表格 (无 head 限制)
    print(f"### 详细数据明细 ({unit})")
    print_markdown_table(final_table)
    
    return fig

dashboard.set_update_function(update_dashboard)
dashboard.show()

## ❄️ 溢额合约全景报告 (冷色系版)

### 详细数据明细 (万元)


| 业务年度   |   保费_超法定自留 |   保费_风险分散 |   损益_超法定自留 |   损益_风险分散 |
|:-----------|------------------:|----------------:|------------------:|----------------:|
| 2001       |            0      |         3.44722 |          0        |         2.01663 |
| 2002       |            6.2836 |       325.303   |          3.67591  |       160.311   |
| 2003       |           25.2821 |       951.194   |         -0.764774 |       448.738   |
| 2004       |           70.5375 |       876.204   |         33.9672   |       432.025   |
| 2005       |          249.355  |       916.032   |        148.644    |       269.163   |
| 2006       |          515.009  |      1529.94    |        227.849    |     -2292.78    |
| 2007       |          518.948  |      2464.65    |        185.569    |       510.483   |
| 2008       |         2294.55   |      3769.46    |        435.917    |      1082.42    |
| 2009       |         3802.4    |      6825.84    |       1380.44     |      2199.12    |
| 2010       |         6737.76   |      8231.6     |      -1126.78     |     -1950.25    |
| 2011       |         7936.01   |      8175.62    |       3159.74     |     -1028.35    |
| 2012       |         8843.17   |      8068.62    |       1517.29     |       267.806   |
| 2013       |         9924.67   |      7683.71    |       -844.618    |     -4175.74    |
| 2014       |        11684.6    |      9632.49    |       4924.43     |       966.355   |
| 2015       |         9804.12   |      9853.5     |      -5734.8      |      2384.88    |
| 2016       |        10568.3    |      8531.67    |       3695.35     |      1773.91    |
| 2017       |        11738.6    |      8798.44    |       -588.83     |       726.923   |
| 2018       |        11296.3    |      9757.56    |       3142.87     |     -1314.07    |
| 2019       |        13074.1    |     11332.4     |       3771.51     |      -451.742   |
| 2020       |        14133.7    |     11344.3     |       1680.3      |      1664.55    |
| 2021       |        14528      |     11042       |       -411.524    |      2131.94    |
| 2022       |        15260.7    |     12883.4     |       2112.74     |      3000.95    |
| 2023       |        18179.9    |     13151.5     |       4952.87     |      1521.55    |
| 2024       |        18725.7    |     14467.1     |       6371.25     |      4733.87    |
| 2025       |        14221.1    |     11432.5     |       5022.15     |      3171.51    |
| **合计**   |       204139      |    182049       |      34059.2      |     16235.6     |

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] FlexBox(objects=[MultiChoice(name='📊 业务年度'...], sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

## 📥 Step 7: 导出 PDF 报告

自动提取 AI 编写的业务备注，生成专业报告。

In [7]:
from src.exporter import ReportExporter
import os
from IPython.display import FileLink
import importlib
import src.exporter, src.dashboard.panel_builder
importlib.reload(src.exporter)
importlib.reload(src.dashboard.panel_builder)
target_obj = globals().get('dashboard')
if target_obj:
    report_path = ReportExporter.export_to_pdf(
        dashboard=target_obj,
        title="数据分析深度报告",
        author="Harold AI 分析助手"
    )
    if report_path and os.path.exists(report_path):
        print("\n✅ 报告生成成功！")
        display(FileLink(report_path))
else:
    print("❌ 错误: 未找到 'dashboard' 对象")


🚀 开始生成 PDF 报告: report_20251231_235731.pdf...
⚠️ 尝试加载 /System/Library/Fonts/Hiragino Sans GB.ttc 失败: TTC file "/System/Library/Fonts/Hiragino Sans GB.ttc": postscript outlines are not supported
✅ 成功加载中文字体: /System/Library/Fonts/STHeiti Medium.ttc

📑 处理仪表盘 1 (📊 数据分析仪表盘 (v3.0.1)):
  🔄 正在执行分析逻辑以提取图表和表格...
## ❄️ 溢额合约全景报告 (冷色系版)

### 详细数据明细 (万元)


| 业务年度   |   保费_超法定自留 |   保费_风险分散 |   损益_超法定自留 |   损益_风险分散 |
|:-----------|------------------:|----------------:|------------------:|----------------:|
| 2001       |            0      |         3.44722 |          0        |         2.01663 |
| 2002       |            6.2836 |       325.303   |          3.67591  |       160.311   |
| 2003       |           25.2821 |       951.194   |         -0.764774 |       448.738   |
| 2004       |           70.5375 |       876.204   |         33.9672   |       432.025   |
| 2005       |          249.355  |       916.032   |        148.644    |       269.163   |
| 2006       |          515.009  |      1529.94    |        227.849    |     -2292.78    |
| 2007       |          518.948  |      2464.65    |        185.569    |       510.483   |
| 2008       |         2294.55   |      3769.46    |        435.917    |      1082.42    |
| 2009       |         3802.4    |      6825.84    |       1380.44     |      2199.12    |
| 2010       |         6737.76   |      8231.6     |      -1126.78     |     -1950.25    |
| 2011       |         7936.01   |      8175.62    |       3159.74     |     -1028.35    |
| 2012       |         8843.17   |      8068.62    |       1517.29     |       267.806   |
| 2013       |         9924.67   |      7683.71    |       -844.618    |     -4175.74    |
| 2014       |        11684.6    |      9632.49    |       4924.43     |       966.355   |
| 2015       |         9804.12   |      9853.5     |      -5734.8      |      2384.88    |
| 2016       |        10568.3    |      8531.67    |       3695.35     |      1773.91    |
| 2017       |        11738.6    |      8798.44    |       -588.83     |       726.923   |
| 2018       |        11296.3    |      9757.56    |       3142.87     |     -1314.07    |
| 2019       |        13074.1    |     11332.4     |       3771.51     |      -451.742   |
| 2020       |        14133.7    |     11344.3     |       1680.3      |      1664.55    |
| 2021       |        14528      |     11042       |       -411.524    |      2131.94    |
| 2022       |        15260.7    |     12883.4     |       2112.74     |      3000.95    |
| 2023       |        18179.9    |     13151.5     |       4952.87     |      1521.55    |
| 2024       |        18725.7    |     14467.1     |       6371.25     |      4733.87    |
| 2025       |        14221.1    |     11432.5     |       5022.15     |      3171.51    |
| **合计**   |       204139      |    182049       |      34059.2      |     16235.6     |

  ✅ 提取完成: 1 个图表, 0 个表格
✅ 报告已成功导出至: outputs/reports/report_20251231_235731.pdf

✅ 报告生成成功！


/Users/harold/working/Jupyter_AI_DataAnalyze/notebooks/02_analysis/outputs/reports/report_20251231_235731.pdf